CARREGA A BASE

In [11]:
import pandas as pd
import numpy as np

# Carregar a base de dados
df = pd.read_csv('pns_2019.csv')

# Base inicial
print(df.shape)  # (linhas, colunas)
copia_df = df.copy()

C:\Users\joaof\AppData\Local\Temp\ipykernel_25076\1225244378.py:5: DtypeWarning: Columns (10,18,21,23,62,63,64,81,84,92,93,101,102,103,110,116,118,122,124,129,197,201,206,214,218,219,229,246,254,298,302,303,304,307,308,309,360,369,379,396,425,432,433,435,436,437,452,457,491,493,501,502,503,504,505,528,534,544,548,561,569,570,571,572,573,574,575,576,578,580,581,582,583,584,585,586,587,588,591,600,608,609,610,611,612,613,614,618,624,625,631,634,635,636,637,638,639,640,641,642,643,644,646,647,648,649,650,651,652,653,654,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,674,675,676,677,678,679,680,694,720,726,727,728,732,734,735,736,741,742,743,744,745,746,747,771,772,773,774,786) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('pns_2019.csv')


(279382, 816)


CARREGA A CÓPIA PARA NÃO PRECISAR FICAR CARREGANDO A BASE REPETIDAS VEZES

In [12]:
df = copia_df.copy()

REMOVE LINHAS ONDE ALVO NÃO FOI RESPONDIDA

In [13]:
df = df.dropna(subset=["[Q088] Algum médico já lhe deu o diagnóstico de DORT? "])
print(df.shape)  # Verificar o novo tamanho do dataframe

(90846, 816)


VERIFICA PROPORÇÃO DE LINHAS PREENCHIDAS POR COLUNA

In [14]:
# Calcular a porcentagem de preenchimento para cada coluna
percent_filled = df.notna().sum() / len(df) * 100

# Contar o número de colunas em cada faixa de preenchimento de 5%, a partir de 50%
ranges = {}
for i in range(50, 100, 5):
    ranges[f'{i}%-{i+5}%'] = percent_filled.between(i, i+5, inclusive="both").sum()

# Para 100%, contagem separada
ranges['100%'] = (percent_filled == 100).sum()

# Exibir os resultados
for range_label, count in ranges.items():
    print(f"{range_label}: {count} colunas")

50%-55%: 3 colunas
55%-60%: 30 colunas
60%-65%: 4 colunas
65%-70%: 2 colunas
70%-75%: 1 colunas
75%-80%: 7 colunas
80%-85%: 11 colunas
85%-90%: 1 colunas
90%-95%: 8 colunas
95%-100%: 239 colunas
100%: 203 colunas


REMOVE COLUNAS COM MENOS DE XX% DOS DADOS PREENCHIDOS

In [15]:
# Calcular o número total de linhas no DataFrame
total_rows = len(df)

# Filtrar colunas com XX% ou mais das linhas preenchidas (sem NaN)
df = df.loc[:, df.notna().sum() >= (0.9 * total_rows)]

# Exibir o novo tamanho do DataFrame
print(df.shape)

(90846, 247)


MANTER APENAS AS COLUNAS RELEVANTES

In [17]:
# Carregar os nomes das colunas relevantes do arquivo, preservando os espaços
with open('colunas_relevantes.txt', 'r', encoding='utf-8') as file:
    colunas_relevantes = [linha.rstrip('\n') for linha in file.readlines()]  # Remove apenas o caractere de nova linha

# Verificar quais colunas do arquivo estão presentes no DataFrame
colunas_existentes = [col for col in colunas_relevantes if col in df.columns]

# Filtrar o DataFrame para manter apenas as colunas existentes
df = df[colunas_existentes]

# Exibir o novo tamanho do DataFrame
print(df.shape)

(90846, 20)


CONVERTE COLUNAS CATEGORICAS DE SIM E NÃO PARA NUMERICAS

In [18]:
cont = 0
for column in df.columns:
    if df[column].dtype == 'object':
        stripped = df[column].astype(str).str.strip()
        unique_values = stripped.dropna().unique()
        if set(unique_values).issubset({'Sim', 'Não'}):
            df[column] = np.where(stripped == 'Sim', 1,
                          np.where(stripped == 'Não', 0, pd.NA))
            cont+=1

print("Colunas convertidas: ", cont)


C:\Users\joaof\AppData\Local\Temp\ipykernel_25076\1398251694.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = np.where(stripped == 'Sim', 1,
C:\Users\joaof\AppData\Local\Temp\ipykernel_25076\1398251694.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = np.where(stripped == 'Sim', 1,
C:\Users\joaof\AppData\Local\Temp\ipykernel_25076\1398251694.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

Colunas convertidas:  7


C:\Users\joaof\AppData\Local\Temp\ipykernel_25076\1398251694.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = np.where(stripped == 'Sim', 1,
C:\Users\joaof\AppData\Local\Temp\ipykernel_25076\1398251694.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = np.where(stripped == 'Sim', 1,


DEFINE O TIPO DA COLUNA COMO FLOAT OU OBJECT

In [19]:
# Converter todas as colunas para string (para análise segura)
df = df.astype(str)

float_columns = []
object_columns = []

for column in df.columns:
    sample_values = df[column].head(100).dropna()

    def is_float_or_empty(val):
        val = val.strip()
        if val == '' or val.lower() in ['nan', 'na']:
            return True
        try:
            float(val)
            return True
        except ValueError:
            return False

    # Verificar se todos os 100 primeiros valores são float-like ou vazios
    all_numeric_or_empty = sample_values.apply(is_float_or_empty).all()

    if all_numeric_or_empty:
        # Converter a coluna completa para float, tratando valores não numéricos como NaN
        df[column] = pd.to_numeric(df[column].str.strip(), errors='coerce')
        float_columns.append(column)
    else:
        df[column] = df[column].astype(object)
        object_columns.append(column)

# Mostrar as colunas classificadas
print(f"\nColunas float ({len(float_columns)}):")
print(float_columns)

print(f"\nColunas object ({len(object_columns)}):")
print(object_columns)



Colunas float (12):
['[Q088] Algum médico já lhe deu o diagnóstico de DORT? ', '[Q084] O(a) Sr(a) tem algum problema crônico de coluna, como dor crônica nas costas ou no pescoço, lombalgia, dor ciática, problemas nas vértebras ou disco? ', '[Q079] Algum médico já lhe deu o diagnóstico de artrite ou reumatismo? ', '[P044] Nas suas atividades domésticas, o(a) Sr(a) faz faxina pesada, carrega peso ou faz outra atividade pesada que requer esforço físico intenso? (não considerar atividade doméstica remunerada) ', '[C008] Idade do morador na data de referência', '[E001] Na semana de___ a___ (semana de referência), ___ trabalhou ou estagiou, durante pelo menos uma hora, em alguma atividade remunerada em dinheiro', '[E033] . Na semana de___a___ (semana de referência), ___ qual foi o total de horas que dedicou às atividades de cuidados de pessoas e/ou afazeres domésticos? ', '[P034] Nos últimos três meses, o(a) Sr(a) praticou algum tipo de exercício físico ou esporte? ', '[J002]  Nas duas últi

PRENCHE DADOS FALTANTES COM A MEDIANA

In [20]:
# Preencher valores faltantes com a mediana para colunas numéricas
for column in df.select_dtypes(include=['float64', 'int64']).columns:
    median_value = df[column].median()
    df[column].fillna(median_value, inplace=True)

print("Dados faltantes preenchidos com a mediana.")

Dados faltantes preenchidos com a mediana.


C:\Users\joaof\AppData\Local\Temp\ipykernel_25076\1459902153.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(median_value, inplace=True)
C:\Users\joaof\AppData\Local\Temp\ipykernel_25076\1459902153.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exampl

USUÁRIO ESCOLHE COMO DESEJA BAIXAR A BASE

In [21]:
entrada = input("Escolha uma das opções abaixo:\n"
                "1. Baixar o relatório tratado como csv\n"
                "2. Baixar o relatório tratado como xlsx\n"
                "3. Baixar amostra do relatório como csv\n"
                "4. Baixar amostra do relatório como xlsx\n"
                "5. Baixar amostra do relatório separado em numerica e categórica\n"
                "6. Baixar txt com os nomes das colunas restantes\n")

In [22]:
if entrada == "1":
    # Baixar a base completa como CSV
    df.to_csv('base_tratada.csv', index=False)
    print("Base completa salva como CSV.")
    
elif entrada == "2":
    # Baixar a base completa como XLSX
    df.to_excel('base_tratada.xlsx', index=False)
    print("Base completa salva como XLSX.")

elif entrada == "3":
    # Manter apenas as primeiras 100 linhas
    df = df.head(100)

    # Baixar a base completa como CSV
    df.to_csv('base_amostra.csv', index=False)
    print("Base completa salva como CSV.")

elif entrada == "4":
    # Manter apenas as primeiras 100 linhas
    df = df.head(100)

    # Baixar a base completa como XLSX
    df.to_excel('base_amostra.xlsx', index=False)
    print("Base completa salva como XLSX.")
    
elif entrada == "5":
    # Manter apenas as primeiras 100 linhas
    df = df.head(100)

    # Salvar as colunas float em uma planilha Excel
    df[float_columns].to_excel('float_columns.xlsx', index=False)
    print("Colunas float salvas em float_columns.xlsx")

    # Salvar as colunas object em uma planilha Excel
    df[object_columns].to_excel('object_columns.xlsx', index=False)
    print("Colunas object salvas em object_columns.xlsx")

elif entrada == "6":
    # Criar um arquivo .txt com os nomes das colunas restantes
    with open("colunas_restantes.txt", "w", encoding="utf-8") as f:
        for coluna in df.columns:
            f.write(coluna + "\n")
    print("Nomes das colunas salvos em colunas_restantes.txt")
else:
    print("Opção inválida. Tente novamente.")


Base completa salva como CSV.
